# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [10]:
prompt = ChatPromptTemplate.from_template(
    "Describe a product called {product} in detail."  # Prompt with a variable placeholder
)


In [11]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_340\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [14]:
product = "electric scooter"  # Choose a product
chain.run(product)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_340\2777155482.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'An electric scooter is a small, lightweight vehicle designed for personal transportation. It typically consists of a narrow platform with two wheels, a handlebar for steering, and an electric motor that provides power for movement. The scooter is powered by a rechargeable battery, which is often located under the platform.\n\nElectric scooters are becoming increasingly popular as a convenient and eco-friendly mode of transportation in urban areas. They are easy to maneuver and can be used to navigate through crowded streets and sidewalks. Many models are foldable, making them easy to carry and store when not in use.\n\nThe top speed of an electric scooter can vary depending on the model, but most can reach speeds of up to 15-25 miles per hour. They typically have a range of around 15-30 miles on a single charge, depending on factors such as rider weight and terrain.\n\nMost electric scooters come equipped with features such as LED lights, a digital display for monitoring battery life 

## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [17]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
        "Describe a product called {product}."

    #Repeat the initial query or create a new query that would feed into the second prompt
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [18]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
        "Now make a social media post using this description: {input}"

    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [19]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The electric scooter is a sleek and portable mode of transportation that runs on battery power. It is designed with a lightweight frame and compact wheels, making it easy to carry and maneuver through crowded streets. The scooter features a powerful electric motor that allows for quick acceleration and smooth cruising speeds. 

The electric scooter is equipped with a rechargeable lithium-ion battery that can be easily plugged in and charged at home or at a charging station. With a fully charged battery, the scooter can travel for several miles before needing to be recharged. 

The scooter also comes with a variety of safety features, such as LED lights for increased visibility at night, a responsive braking system, and a sturdy frame for stability while riding. The electric scooter is a convenient and eco-friendly mode of transportation for urban commuters and travelers looking for a fun and efficient way to get around.
Check out the slee

'Check out the sleek and portable electric scooter - perfect for navigating crowded streets with its lightweight frame and compact design. With a powerful electric motor and rechargeable battery, this scooter offers quick acceleration and smooth cruising speeds. Stay safe with LED lights, a responsive braking system, and sturdy frame. Convenient, eco-friendly, and fun - the perfect ride for urban commuters and travelers! 🛴🔋 #ElectricScooter #EcoFriendlyTransportation #UrbanCommute'

**Repeat the above twice for different products**

## SequentialChain

In [21]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template("Translate this review to Arabic: {Review}")

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="arabic_review")


In [ ]:
second_prompt = ChatPromptTemplate.from_template("Summarize this review: {Review}"
    #Write a promplt to summarize a review
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [26]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template("Which language is this review in? {Review}")


# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [27]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Send a follow-up message based on the summary: {summary} and language: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="message")



In [28]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["arabic_review", "summary", "language", "message"],
    verbose=True
)

In [29]:
review = df.Review[5]
overall_chain(review)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_340\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'arabic_review': 'أجد الطعم سيء. لا تدوم الرغوة ، هذا غريب. أشتري نفس المنتج في المتاجر والطعم أفضل بكثير ... هل يعود الأمر إلى دفعة قديمة أم تقليد؟',
 'summary': "L'auteur trouve le goût médiocre de la boisson et remarque que la mousse ne tient pas bien, ce qui est inhabituel. Ils pensent que le produit pourrait être un vieux lot ou une contrefaçon, car ils achètent habituellement le même produit dans le commerce avec un goût bien meilleur.",
 'language': 'French',
 'message': "Bonjour,\n\nJe suis désolé d'entendre que vous avez eu une mauvaise expérience avec la boisson. Il est possible que vous ayez reçu un vieux lot ou une contrefaçon. Je vous conseille de contacter le fabricant pour signaler le problème et peut-être obtenir un remboursement ou un remplacement. J'espère que votre prochaine expérience sera 

**Repeat the above twice for different products or reviews**

## Router Chain

In [30]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [31]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [32]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [33]:
llm = ChatOpenAI(temperature=0)

In [34]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [35]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [36]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [37]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [38]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_340\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [39]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [40]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [41]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and for passing on genetic traits to future generations. \n\nAdditionally, DNA serves as a blueprint for the regulation of gene expression, which determines how genes are turned on or off in response to different environmental cues. This allows our cells to adapt and respond to changes in their surroundings, ensuring the proper functioning of our bodies. \n\nIn summary, the presence of DNA in every cell is essen

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [42]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.7)

prompt = ChatPromptTemplate.from_template(
    "Describe a product called {product} in detail."
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "Bluetooth Speaker"
print(chain.run(product))


A Bluetooth speaker is a portable device that allows you to stream music from your smartphone, tablet, or other Bluetooth-enabled devices wirelessly. It is equipped with built-in Bluetooth technology that allows for a seamless connection to your device without the need for any cords or cables.

Bluetooth speakers come in a variety of shapes, sizes, and designs, ranging from compact and lightweight models to larger, more powerful options. They are typically powered by a rechargeable battery, allowing you to take your music with you on the go.

Many Bluetooth speakers offer additional features such as water resistance, voice control, and built-in microphone for hands-free calling. Some models also come with LED lights, EQ settings, and the ability to connect multiple speakers for a surround sound experience.

The sound quality of Bluetooth speakers can vary, but most models offer impressive audio performance with clear vocals, deep bass, and crisp highs. Some high-end models even come wi

In [43]:
from langchain.chains import SimpleSequentialChain

llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Describe a product called {product}."
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template(
    "Write an Instagram caption for this product description: {input}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

print(overall_simple_chain.run("Smart Coffee Mug"))
print(overall_simple_chain.run("Electric Toothbrush"))




> Entering new SimpleSequentialChain chain...
The Smart Coffee Mug is a cutting-edge, technologically advanced mug that is designed to enhance your coffee drinking experience. This mug is equipped with built-in sensors and a digital display that allows you to customize and control the temperature of your drink with precision.

The Smart Coffee Mug features a sleek, modern design and is made of high-quality, durable materials. It is also insulated to keep your hot beverages hot and your cold beverages cold for an extended period of time.

With the Smart Coffee Mug, you can say goodbye to lukewarm coffee or tea. Simply set your desired temperature on the digital display, and the mug will automatically heat or cool your drink to that perfect temperature. You can even save your preferred settings for future use.

In addition, the Smart Coffee Mug can connect to your smartphone via Bluetooth, allowing you to track your caffeine intake, receive drink recommendations, and even order coffee 

In [46]:
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template("Translate this review to French: {Review}")
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="french_review")

second_prompt = ChatPromptTemplate.from_template("Summarize this review: {Review}")
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template("Which language is this review in? {Review}")
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Send a follow-up message in {language} based on this summary: {summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="message")

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["french_review", "summary", "language", "message"],
    verbose=True
)

if len(df) >= 3:
    review_1 = df.Review.iloc[0]
    review_2 = df.Review.iloc[1]
    review_3 = df.Review.iloc[2]
else:
    print("Not enough reviews to run all chains.")


print(overall_chain(review_1))
print(overall_chain(review_2))
print(overall_chain(review_3))




> Entering new SequentialChain chain...

> Finished chain.
{'Review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.', 'french_review': "J'ai commandé un ensemble king size. Ma seule critique serait que je souhaite que le vendeur propose l'ensemble king size avec 4 taies d'oreiller. J'ai commandé séparément un lot de deux taies d'oreiller pour en avoir un total de quatre. Lorsque j'ai vu les deux paquets, il semblait que la couleur ne correspondait pas exactement. Le service client a été excellent en m'envoyant deux taies d'oreiller supplémentai

In [47]:
chain.run("What is black body radiation?")
chain.run("what is 2 + 2")
chain.run("Why does every cell in our body contain DNA?")
chain.run("Explain the impact of World War 2 on global politics")
chain.run("How does recursion work in programming?")


'"How does recursion work in programming?" is a software tool designed to help beginners understand and master the concept of recursion in programming. Recursion is a fundamental programming technique in which a function calls itself in order to solve a problem. This tool provides a comprehensive explanation of how recursion works, along with examples and exercises to reinforce learning.\n\nThe product features a user-friendly interface that guides users through the basics of recursion, including the concept of base cases, recursive cases, and the call stack. It also covers common pitfalls and best practices when using recursion in programming.\n\nIn addition to theoretical explanations, "How does recursion work in programming?" also offers interactive coding challenges where users can practice writing recursive functions in various programming languages, such as Python, Java, and C++. These challenges are designed to gradually increase in difficulty, allowing users to build their skil